# Install & Import packages

In [1]:
import torch
import torch.nn as nn

In [2]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
# ignore warnings
import warnings ; warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import normalize
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence


# Encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#pickle
import pickle

In [2]:
# 설치된 글꼴 이름 파악을 위해서!!
import matplotlib as mpl
from matplotlib import font_manager as fm
from matplotlib import rc

# 설정 파일 위치 확인
print(mpl.matplotlib_fname())
print(mpl.__file__)

# Jupyter Notebook 로드 시 같이 올라온 font 목록 확인
[f.fname for f in fm.fontManager.ttflist]	# NanumGothic.ttf 목록에서 확인 완료!!!

# NanumGothic 설정 확인
plt.rcParams['font.family'] = 'NanumGothic'
print(plt.rcParams['font.family'])		# NanumGothic 설정 제대로 먹힌 것을 확인!!!

# 마이너스(-)가 '□'로 깨지는 것을 방지하기 위하여 설정!!!
mpl.rcParams['axes.unicode_minus'] = False


/home/ubuntu/anaconda3/envs/venv/lib/python3.10/site-packages/matplotlib/mpl-data/matplotlibrc
/home/ubuntu/anaconda3/envs/venv/lib/python3.10/site-packages/matplotlib/__init__.py
['NanumGothic']


In [3]:
def summary(df):
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index':'피쳐'})
    summary['결측값 개수'] = df.isnull().sum().values
    summary['고유값 개수'] = df.nunique().values
    summary['첫번째 값'] = df.loc[0].values
    summary['두번째 값'] = df.loc[1].values
    return summary

## Import Data

In [34]:
df = pd.read_csv('1004_df.csv', low_memory=False)

In [35]:
# 청장년층으로 필터링
df = df[df['age'].isin(['20e', '20s', '30e', '30s'])]

In [36]:
df.date = df.date.astype('str')

In [37]:
# 'year'와 'month' 컬럼 생성
df['year'] = df['date'].str[:2] 
df['month'] = df['date'].str[2:]  

In [38]:
df.year= df.year.astype('int')
df.month= df.month.astype('int')
df.date = df.date.astype('int')

In [39]:
# 2개월 전의 값으로 2개월 후의 lb08m을 설명하는 통계 모델을 만들고 예측값을 지표로 사용하고자 한다.

ease = df.query('date > 2208')[['lb08m','month','date','year','gender','age','region']].copy() 
ease['month'] = (ease['month'] - 2).astype(str).str.zfill(2)  # 한 자리수 월을 두 자리수로 만듭니다.

ease['year'] = ease['year'].astype(str)
ease['date'] = ease['year'] + ease['month']  # 'year'와 'month'를 합쳐 'date'를 만듭니다.


In [40]:
ease['date'] = ease['date'].replace('2300', '2212')

In [41]:
ease = ease.drop(['year','month'],axis=1)

In [42]:
df.date = df.date.astype('str')

In [43]:
Y = df.query('date=="2306"')

In [44]:
y = df['lb08m']

In [45]:
df.date=df.date.astype('int')
ease.date=ease.date.astype('int')

In [46]:
df = df.query('date<2306').drop(['lb08m','year','month'],axis=1).merge(ease, on =['date','gender','age','region'])

## Encoding

In [47]:
# LabelEncoder를 객체로 생성
encoder = LabelEncoder()

# fit, transform 메소드를 통한 레이블 인코딩
encoder.fit(df['age'])

df["age"] = encoder.transform(df['age'])


Y["age"] = encoder.transform(Y['age'])

In [48]:
#성별
oh = OneHotEncoder()
encoder = oh.fit_transform(df['gender'].values.reshape(-1,1)).toarray() # 인코딩 하기 전에 2차원 데이터로 변환
df_OneHot = pd.DataFrame(encoder, columns=["gender_" + str(oh.categories_[0][i]) for i in range (len(oh.categories_[0]))])

df = pd.concat([df, df_OneHot], axis=1)

# test 데이터에 transform
encoder = oh.transform(Y['gender'].values.reshape(-1, 1)).toarray()
df_test_OneHot = pd.DataFrame(encoder, columns=["gender_" + str(oh.categories_[0][i]) for i in range (len(oh.categories_[0]))])
Y = pd.concat([Y.reset_index(drop=True), df_test_OneHot], axis=1).drop(columns=['gender'])

In [49]:
df.drop(columns = ['gender','광역시도명','구명','행정동명','시군구명'], inplace=True)
Y.drop(columns = ['광역시도명','구명','행정동명','시군구명'], inplace=True)

In [50]:
df.date.unique()

array([2208, 2210, 2212, 2302, 2304])

In [51]:
df = df[df.notnull().all(axis=1)]
print(df.isna().sum().sum())

Y=Y[Y.notnull().all(axis=1)]
print(Y.isna().sum().sum())

0
0


In [52]:
Y

,date,age,region,ba01m,ba02m,ba03r,ba04m,ba05r,ba06m,ba07r,...,lf02r,lf03r,lf04r,lf05r,lf06r,lf07r,year,month,gender_F,gender_M
0,2306,0,11110515,11840000,5672000,0.01,31696000,0.52,7731000,0.07,...,0.72,0.24,0.15,0.19,0.46,0.20,23,6,1.0,0.0
1,2306,1,11110515,4401000,1193000,0.00,0,0.50,4838000,0.04,...,0.64,0.25,0.24,0.16,0.38,0.22,23,6,1.0,0.0
2,2306,2,11110515,14520000,3509000,0.02,16000,0.47,6770000,0.12,...,0.72,0.24,0.11,0.21,0.50,0.18,23,6,1.0,0.0
3,2306,3,11110515,12624000,5995000,0.02,814000,0.50,7845000,0.07,...,0.75,0.23,0.11,0.20,0.52,0.18,23,6,1.0,0.0
4,2306,0,11110515,7272000,3648000,0.01,1006000,0.43,5693000,0.03,...,0.84,0.23,0.05,0.12,0.63,0.20,23,6,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7626,2306,2,41590600,11767000,3752000,0.02,4497000,0.44,4993000,0.11,...,0.82,0.27,0.00,0.09,0.82,0.09,23,6,1.0,0.0
7634,2306,2,41610253,8568000,3423000,0.01,1673000,0.37,4051000,0.08,...,0.56,0.13,0.19,0.38,0.38,0.06,23,6,1.0,0.0
7635,2306,3,41610253,7516000,1679000,0.00,3000,0.47,6494000,0.09,...,0.90,0.20,0.10,0.20,0.60,0.10,23,6,1.0,0.0
7638,2306,2,41610253,8370000,4374000,0.00,285000,0.34,4097000,0.04,...,0.56,0.06,0.25,0.31,0.38,0.06,23,6,0.0,1.0


In [53]:
columns =  [i for i in df.columns if i not in ['date', 'region','age','gender_F', 'gender_M']]

In [54]:
Y.drop(['month','year'],axis=1, inplace=True)

In [55]:
df.head()

,date,age,region,ba01m,ba02m,ba03r,ba04m,ba05r,ba06m,ba07r,...,lf01r,lf02r,lf03r,lf04r,lf05r,lf06r,lf07r,lb08m,gender_F,gender_M
0,2208,0,11110515,9378000,3775000,0.01,333000,0.57,6153000,0.08,...,0.75,0.71,0.24,0.15,0.19,0.48,0.19,99000.0,1.0,0.0
1,2208,1,11110515,4836000,1784000,0.00,0,0.51,3587000,0.07,...,0.67,0.59,0.19,0.26,0.18,0.41,0.15,113000.0,1.0,0.0
2,2208,2,11110515,14055000,4079000,0.02,104000,0.44,9411000,0.07,...,0.74,0.69,0.23,0.14,0.23,0.45,0.17,96000.0,1.0,0.0
3,2208,3,11110515,9092000,4322000,0.02,2700000,0.54,5482000,0.03,...,0.78,0.72,0.23,0.12,0.21,0.50,0.17,100000.0,1.0,0.0
4,2208,0,11110515,5515000,2098000,0.01,986000,0.46,5425000,0.01,...,0.93,0.83,0.26,0.04,0.13,0.60,0.23,116000.0,0.0,1.0


In [56]:
Y.head()

,date,age,region,ba01m,ba02m,ba03r,ba04m,ba05r,ba06m,ba07r,...,le14r,lf01r,lf02r,lf03r,lf04r,lf05r,lf06r,lf07r,gender_F,gender_M
0,2306,0,11110515,11840000,5672000,0.01,31696000,0.52,7731000,0.07,...,0.05,0.75,0.72,0.24,0.15,0.19,0.46,0.20,1.0,0.0
1,2306,1,11110515,4401000,1193000,0.00,0,0.50,4838000,0.04,...,0.13,0.70,0.64,0.25,0.24,0.16,0.38,0.22,1.0,0.0
2,2306,2,11110515,14520000,3509000,0.02,16000,0.47,6770000,0.12,...,0.10,0.78,0.72,0.24,0.11,0.21,0.50,0.18,1.0,0.0
3,2306,3,11110515,12624000,5995000,0.02,814000,0.50,7845000,0.07,...,0.05,0.78,0.75,0.23,0.11,0.20,0.52,0.18,1.0,0.0
4,2306,0,11110515,7272000,3648000,0.01,1006000,0.43,5693000,0.03,...,0.05,0.90,0.84,0.23,0.05,0.12,0.63,0.20,0.0,1.0


In [57]:
df.drop(columns = ['lb08m'], inplace=True)

In [58]:
target=Y.lb08m

In [59]:
y_train=df.lb08m

In [60]:
Y.drop(columns = ['lb08m'], inplace=True)

In [62]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# train 데이터에 fit_transform
X_train_scaled = pd.DataFrame(scaler.fit_transform(df.iloc[:,3:-2]), columns=df.iloc[:,3:-2].columns)

# test 데이터에 transform
X_test_scaled = pd.DataFrame(scaler.transform(Y.iloc[:,3:-2]), columns=Y.iloc[:,3:-2].columns)

print("Train Data:")
print(X_train_scaled)

print("\nTest Data:")
print(X_test_scaled)

In [242]:
temp=pd.DataFrame(X_train_scaled)
temp_test=pd.DataFrame(X_test_scaled)

In [243]:
temp = pd.concat([df.loc[:,['date', 'age', 'region', 'gender_F', 'gender_M']].reset_index(drop=True),temp], axis=1)

In [244]:
temp_test =  pd.concat([Y.loc[:,['date', 'age', 'region', 'gender_F', 'gender_M']].reset_index(drop=True),temp_test], axis=1)

In [245]:
df = temp.copy()

In [246]:
X_test=temp_test.copy()

## 상관행렬 계산

In [247]:
# 상관행렬 계산
correlation_matrix = df.corr().abs()

# 상관계수가 0.8 이상인 변수들을 선택
high_corr_var = np.where(correlation_matrix > 0.9)

# (i, j) 인덱스 튜플의 리스트 생성
high_corr_var = [(correlation_matrix.columns[x], correlation_matrix.columns[y]) for x, y in zip(*high_corr_var) if x != y and x < y]

# 상관계수가 높은 변수들을 제거
for var_pair in high_corr_var:
    df.drop(var_pair[1], axis=1, inplace=True)  # 두 번째 변수를 제거


In [248]:
print(high_corr_var)

[('gender_F', 'gender_M'), ('bb01r', 'bd21r'), ('bc02r', 'bd23r'), ('bd07r', 'bd11r'), ('bd27r', 'bd29r'), ('be01r', 'be04r'), ('be02r', 'be06r'), ('cc03m', 'cc16m'), ('ce14m', 'ce29m'), ('cg23m', 'cg24m'), ('ch01r', 'ch04r'), ('sa01r', 'sa02r'), ('sa01r', 'sa19r'), ('sa05r', 'sa06r'), ('sb10r', 'sc10r'), ('sb16r', 'sb20r')]


In [249]:
df.shape

(11824, 417)

In [250]:
bank = df.loc[:, ['date', 'gender_F','age', 'region'] + list(df.loc[:, 'ba01m':'be07r'].columns)]
card = df.loc[:, ['date', 'gender_F',  'age', 'region'] + list(df.loc[:, 'ca01m':'ch07r'].columns)]
stock = df.loc[:, ['date', 'gender_F', 'age', 'region'] + list(df.loc[:, 'sa01r':'sd07r'].columns)]
life = df.loc[:, ['date', 'gender_F',  'age', 'region'] + list(df.loc[:, 'la01r':'lf07r'].columns)]

## 단계선택법

In [251]:
def stepwise_feature_selection(X, y):
    variables=X.columns.tolist()
    steps = []
    sv_per_step, adjusted_r_squared = [], [] ## 각 스텝별로 선택된 변수들, 수정된 결정계수
    selected_variables = [] ## 선택된 변수들
    
    step = 0
    while len(variables) > 0:
        remainder = list(set(variables) - set(selected_variables))
        pval = pd.Series(index=remainder) 
        for col in remainder: 
            X_train = X[selected_variables+[col]]
            X_train = sm.add_constant(X_train)
            model = sm.OLS(y,X_train).fit(disp=0)
            pval[col] = model.pvalues[col]
    
        min_pval = pval.min()
        if min_pval < 0.05: 
            selected_variables.append(pval.idxmin())
            while len(selected_variables) > 0:
                selected_X = X[selected_variables]
                selected_X = sm.add_constant(selected_X)
                selected_pval = sm.OLS(y,selected_X).fit(disp=0).pvalues.iloc[1:]
                max_pval = selected_pval.max()
                if max_pval >= 0.05:   ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                    remove_variable = selected_pval.idxmax()
                    selected_variables.remove(remove_variable)
                else:
                    break
            
            step += 1
            steps.append(step)
            adj_r_squared = sm.OLS(y,sm.add_constant(X[selected_variables])).fit(disp=0).rsquared_adj
            adjusted_r_squared.append(adj_r_squared)
            sv_per_step.append(selected_variables.copy())
        else:
            break
    return selected_variables

In [252]:
def LRC(model):   
    coef = model.params
    LRC_table = pd.DataFrame({'feature_name': coef.index, 'LRC': np.abs(coef.values)/np.sum(np.abs(coef.values))})
    LRC_table['rank'] = LRC_table['LRC'].rank(method='average', ascending=True)
    return LRC_table

In [253]:
feature_bank = [i for i in bank.columns if i not in ['date']]
feature_card = [i for i in card.columns if i not in ['date']]
feature_stock = [i for i in stock.columns if i not in ['date']]
feature_life = [i for i in life.columns if i not in ['date']]

def Weight(LRC_table):
    for i in [('은행', feature_bank), ('카드', feature_card), ('투자', feature_stock),('라이프',feature_life)]:
        print(f'{i[0]} variable:', LRC_table.query('feature_name in @i[1]')['feature_name'].values)
        print(f'{i[0]}:', (LRC_table.query('feature_name in @i[1]')['rank'].sum()/sum(range(1,LRC_table.shape[0]+1))).round(3))

In [259]:
# X는 독립변수, y는 종속변수입니다.
X = df.iloc[:,1:]
y = y_train

# RandomForestRegressor

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 피처 중요도 추출
importances = model.feature_importances_

# 피처 중요도를 데이터프레임으로 변환
feature_importances = pd.DataFrame({'feature': X_train.columns, 'importance': importances})

# 중요도에 따라 피처를 정렬
feature_importances = feature_importances.sort_values(by='importance', ascending=False)


In [119]:
# 중요도가 특정 임계값보다 낮은 피처 제거
selected_features = feature_importances[feature_importances['importance'] > 0.02]['feature']
selected_features

0     gender_F
2       region
49       bd09r
51       bd12m
24       bb04m
26       bb06m
10       ba08m
Name: feature, dtype: object

In [120]:
# 중요도가 특정 임계값보다 낮은 피처 제거
X_selected = X[selected_features]

In [121]:
X_selected
#성별
#지역
#유동거래_체크카드비율 
#유동거래_보험료금액 
#대출_일반대출평잔
#대출_신용대출평잔
#수신_예금평잔

,gender_F,region,bd09r,bd12m,bb04m,bb06m,ba08m
0,0.012842,11110515,0.011959,0.006199,0.005908,0.002475,0.005106
1,0.012842,11110515,0.014267,0.004006,0.000305,0.007933,0.003425
2,0.012842,11110515,0.009651,0.011808,0.011978,0.009322,0.009029
3,0.012842,11110515,0.010701,0.006663,0.001646,0.010705,0.003217
4,0.000000,11110515,0.010491,0.008518,0.001793,0.004854,0.003391
...,...,...,...,...,...,...,...
11819,0.000000,41590259,0.010491,0.007886,0.006901,0.007480,0.007871
11820,0.000000,41590600,0.006714,0.010964,0.012443,0.017870,0.006926
11821,0.000000,41590600,0.005875,0.009910,0.010202,0.017120,0.006715
11822,0.000000,41590600,0.006714,0.009404,0.007654,0.016706,0.013509


In [122]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [123]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 0.0014806517234964044


# MultiLinearRegression

In [260]:
df.date.unique()
# 2208 데이터로 2210 예측
# 2210데이터로 2212 예측
# 2302데이터로 2304 예측

array([2208, 2210, 2302])

In [264]:

X = X.reset_index(drop=True)
y = y.reset_index(drop=True)


In [289]:
# 선택된 변수를 사용하여 모델을 만듭니다.
selected_variables = stepwise_feature_selection(X, y)
formula = "{} ~ {}".format(y.name, ' + '.join(selected_variables))
model_GT = sm.OLS.from_formula(formula, data=pd.concat([df,y], axis=1)).fit()
model_GT.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lb08m   R-squared:                       0.408
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     115.9
Date:                Fri, 06 Oct 2023   Prob (F-statistic):               0.00
Time:                        09:34:46   Log-Likelihood:            -1.2860e+05
No. Observations:               11824   AIC:                         2.573e+05
Df Residuals:                   11753   BIC:                         2.579e+05
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   9.563e+04    287.568    332.535      0.000    9.51e+04    9.62e+04
le09r       2655.5421    206.673     12.849      0.000    2250.428    3060.656
le01r       1277.1695    186.318      6.855      0.000     911.954    1642.385
lb13m       2884.5701    130.285     22.141      0.000    2629.191    3139.950
gender_F   -4490.9525    511.309     -8.783      0.000   -5493.203   -3488.702
lb02r      -1974.9685    222.109     -8.892      0.000   -2410.339   -1539.598
lb14m       1785.3853    138.302     12.909      0.000    1514.290    2056.481
lb09m       2496.4468    154.564     16.152      0.000    2193.476    2799.418
lb04r       -777.4341    200.581     -3.876      0.000   -1170.605    -384.263
cf23r       -391.1505    164.568     -2.377      0.017    -713.731     -68.570
lb06r       1945.8930    175.077     11.115      0.000    1602.714    2289.072
be03r       -605.9202    224.438     -2.700      0.007   -1045.855    -165.985
la06r        759.8706    126.498      6.007      0.000     511.914    1007.827
lb05r       1954.3890    205.083      9.530      0.000    1552.392    2356.386
lb01r       1537.7461    215.793      7.126      0.000    1114.757    1960.735
lb10m        695.5131    123.898      5.614      0.000     452.652     938.374
le03r       -757.2030    168.889     -4.483      0.000   -1088.254    -426.152
ba17r        324.9783    143.324      2.267      0.023      44.039     605.917
cf14m       -353.3333    135.616     -2.605      0.009    -619.163     -87.504
lc05r       -412.4531    126.180     -3.269      0.001    -659.786    -165.120
sc18r        402.2464    129.256      3.112      0.002     148.883     655.610
ce38r        389.9012    178.848      2.180      0.029      39.329     740.473
ld06r       -479.4686    126.234     -3.798      0.000    -726.908    -232.029
bd12m        390.8562    137.522      2.842      0.004     121.290     660.422
ld04r       -309.3693    124.909     -2.477      0.013    -554.212     -64.527
le08r        455.9176    137.644      3.312      0.001     186.113     725.722
cg14m       -354.7620    123.938     -2.862      0.004    -597.701    -111.823
cg46r        427.7122    144.736      2.955      0.003     144.005     711.419
ce16m       -332.0054    122.734     -2.705      0.007    -572.584     -91.426
ba11r        538.6999    164.622      3.272      0.001     216.014     861.386
bb08m       -515.6945    142.450     -3.620      0.000    -794.920    -236.469
sb27r        543.6614    141.083      3.853      0.000     267.116     820.207
sa17r        416.9484    169.855      2.455      0.014      84.005     749.892
cc30r       -450.1198    150.645     -2.988      0.003    -745.409    -154.831
cd10r        339.1464    154.309      2.198      0.028      36.674     641.619
cf19r        332.4506    122.257      2.719      0.007      92.807     572.094
be

In [290]:
# 모델 저장
with open('MultiLinearRegression_1006.pkl', 'wb') as f:
    pickle.dump(model_GT, f)


In [291]:
# 분야별 가중치 산정
# 분야별 가중치를 산출한다.
Weight(LRC(model_GT))

은행 variable: ['gender_F' 'be03r' 'ba17r' 'bd12m' 'ba11r' 'bb08m' 'be05r' 'bb10m']
은행: 0.141
카드 variable: ['gender_F' 'cf23r' 'cf14m' 'ce38r' 'cg14m' 'cg46r' 'ce16m' 'cc30r'
 'cd10r' 'cf19r' 'cg61r' 'cc01m' 'cf04m' 'cg30m' 'cg44m' 'ce28m' 'ce06m'
 'ce21m' 'ce17m' 'cg05m' 'cc31r' 'cc10o' 'cg07m' 'ce10m' 'ce36r' 'cc33r']
카드: 0.271
투자 variable: ['gender_F' 'sc18r' 'sb27r' 'sa17r' 'sb09m' 'sa23r' 'sc12r' 'sb36r'
 'sb40r' 'sb26m' 'sa03r' 'sb11r' 'sa14r']
투자: 0.159
라이프 variable: ['le09r' 'le01r' 'lb13m' 'gender_F' 'lb02r' 'lb14m' 'lb09m' 'lb04r'
 'lb06r' 'la06r' 'lb05r' 'lb01r' 'lb10m' 'le03r' 'lc05r' 'ld06r' 'ld04r'
 'le08r' 'ld09m' 'lc04r' 'lc07r' 'ld05r' 'lb11m' 'lb03r' 'lf03r' 'lf05r']
라이프: 0.484


In [40]:
feature_set = ['gender_F','be03r','ba17r','bd12m','ba11r','bb08m','be05r','bb10m', 'cf23r' ,'cf14m' ,'ce38r' ,'cg14m','cg46r', 'ce16m' ,'cc30r',
 'cd10r' ,'cf19r' ,'cg61r', 'cc01m' ,'cf04m' ,'cg30m', 'cg44m', 'ce28m' ,'ce06m',
 'ce21m', 'ce17m' ,'cg05m', 'cc31r' ,'cc10o' ,'cg07m', 'ce10m', 'ce36r' ,'cc33r', 'sc18r', 'sb27r' ,'sa17r' ,'sb09m' ,'sa23r' ,'sc12r' ,'sb36r'
 ,'sb40r', 'sb26m' ,'sa03r', 'sb11r', 'sa14r','le09r' ,'le01r', 'lb13m',  'lb02r', 'lb14m','lb09m' ,'lb04r'
 ,'lb06r', 'la06r' ,'lb05r', 'lb01r', 'lb10m' ,'le03r', 'lc05r' ,'ld06r' ,'ld04r'
, 'le08r', 'ld09m' ,'lc04r', 'lc07r', 'ld05r' ,'lb11m', 'lb03r', 'lf03r' ,'lf05r']

In [41]:
feature_manual = pd.read_csv('데이터정의.csv')

In [319]:
model_GT.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lb08m   R-squared:                       0.408
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     115.9
Date:                Fri, 06 Oct 2023   Prob (F-statistic):               0.00
Time:                        10:02:03   Log-Likelihood:            -1.2860e+05
No. Observations:               11824   AIC:                         2.573e+05
Df Residuals:                   11753   BIC:                         2.579e+05
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   9.563e+04    287.568    332.535      0.000    9.51e+04    9.62e+04
le09r       2655.5421    206.673     12.849      0.000    2250.428    3060.656
le01r       1277.1695    186.318      6.855      0.000     911.954    1642.385
lb13m       2884.5701    130.285     22.141      0.000    2629.191    3139.950
gender_F   -4490.9525    511.309     -8.783      0.000   -5493.203   -3488.702
lb02r      -1974.9685    222.109     -8.892      0.000   -2410.339   -1539.598
lb14m       1785.3853    138.302     12.909      0.000    1514.290    2056.481
lb09m       2496.4468    154.564     16.152      0.000    2193.476    2799.418
lb04r       -777.4341    200.581     -3.876      0.000   -1170.605    -384.263
cf23r       -391.1505    164.568     -2.377      0.017    -713.731     -68.570
lb06r       1945.8930    175.077     11.115      0.000    1602.714    2289.072
be03r       -605.9202    224.438     -2.700      0.007   -1045.855    -165.985
la06r        759.8706    126.498      6.007      0.000     511.914    1007.827
lb05r       1954.3890    205.083      9.530      0.000    1552.392    2356.386
lb01r       1537.7461    215.793      7.126      0.000    1114.757    1960.735
lb10m        695.5131    123.898      5.614      0.000     452.652     938.374
le03r       -757.2030    168.889     -4.483      0.000   -1088.254    -426.152
ba17r        324.9783    143.324      2.267      0.023      44.039     605.917
cf14m       -353.3333    135.616     -2.605      0.009    -619.163     -87.504
lc05r       -412.4531    126.180     -3.269      0.001    -659.786    -165.120
sc18r        402.2464    129.256      3.112      0.002     148.883     655.610
ce38r        389.9012    178.848      2.180      0.029      39.329     740.473
ld06r       -479.4686    126.234     -3.798      0.000    -726.908    -232.029
bd12m        390.8562    137.522      2.842      0.004     121.290     660.422
ld04r       -309.3693    124.909     -2.477      0.013    -554.212     -64.527
le08r        455.9176    137.644      3.312      0.001     186.113     725.722
cg14m       -354.7620    123.938     -2.862      0.004    -597.701    -111.823
cg46r        427.7122    144.736      2.955      0.003     144.005     711.419
ce16m       -332.0054    122.734     -2.705      0.007    -572.584     -91.426
ba11r        538.6999    164.622      3.272      0.001     216.014     861.386
bb08m       -515.6945    142.450     -3.620      0.000    -794.920    -236.469
sb27r        543.6614    141.083      3.853      0.000     267.116     820.207
sa17r        416.9484    169.855      2.455      0.014      84.005     749.892
cc30r       -450.1198    150.645     -2.988      0.003    -745.409    -154.831
cd10r        339.1464    154.309      2.198      0.028      36.674     641.619
cf19r        332.4506    122.257      2.719      0.007      92.807     572.094
be

In [321]:
feature_manual.query('컬럼ID in @feature_set')[['컬럼ID','컬럼설명']].컬럼설명.unique()

array(['펀드계좌 보유 비율', '청약 보유 비율', '평균 주택대출평잔', '평균 종합통장대출 한도 금액',
       '유동성거래 중 보험료 평균 지급금액', '신한은행 외에 신한라이프를 함께 거래하는 고객의 비율',
       '신한은행 외에 신한카드/신한투자증권/신한라이프 중 1개 그룹사와 거래하는 고객의 비율', '백화점 평균 이용금액',
       '신차(수입차 제외) 평균 이용금액을 1,2,3,4,5로 범주화. 수치가 5에 가까울수록 평균 이용금액이 많고, 1에 가까울수록 이용금액이 적음을 의미함',
       '이커머스(쿠팡, 11번가 등) 이용비율', '면세점 이용비율', '화장품 이용비율',
       '대형고가커피(스타벅스 등) 이용비율', '통신요금 평균 이용금액', '세탁소 평균 이용금액',
       '고속버스 평균 이용건수', '택시 평균 이용건수', '통신요금 평균 이용건수', '주유소 평균 이용건수',
       '통신요금 이용비율', '네컷 즉석사진(인생네컷 등) 이용비율', '약국 평균 이용금액',
       '납골/묘지, 장례식장, 상조서비스 평균 이용건수', '치과 이용비율', '생명보험, 손해보험 이용비율',
       '동해 지역(강릉시, 동해시, 삼척시, 속초시, 고성군, 양양군)에서 결제한 평균 이용금액',
       '스포츠센터(피트니스, 필라테스 등) 평균 이용금액', '놀이동산 평균 이용금액', '수영장 평균 이용건수',
       '해외 관련(여행, 직구 등) 평균 이용건수', '호텔(특급호텔 제외), 콘도, 펜션 등 숙박 이용비율',
       '예술품, 골동품, 화랑, 표구사 이용비율',
       '계좌개설 이후 기준년월 시점까지 증권의 해외주식 거래경험이 있는 고객의 비율',
       '계좌개설 이후 기준년월 시점까지 증권의 신용서비스 거래경험이 있는 고객의 비율',
       '계좌개설 이후 기준년월 시점까지 증권 거래경험 상품이 2개인 고객의 비율(상품구분 : 국내

In [298]:
from sklearn.metrics import mean_squared_error, r2_score
# 테스트 데이터로 예측을 수행합니다.
predictions = model_GT.predict(X_test)

# 3. 예측된 값과 실제 값의 차이를 통해 모델의 성능을 평가합니다.

# RMSE 계산
rmse = np.sqrt(mean_squared_error(target, predictions))

# R-squared 계산
r2 = r2_score(target, predictions)

print(f'RMSE: {rmse}')
print(f'R-squared: {r2}')


RMSE: 10286.627542867562
R-squared: 0.6689087110833772


In [316]:
predictions.describe()

count      5463.000000
mean      92763.340870
std       10846.508414
min       49434.041026
25%       85741.142788
50%       92801.787987
75%       99598.228214
max      145502.545229
dtype: float64

In [317]:
y_train.describe()

count     11824.000000
mean      93323.240866
std       16636.733435
min       34000.000000
25%       83000.000000
50%       92000.000000
75%      102000.000000
max      218000.000000
Name: lb08m, dtype: float64

In [304]:
# 모델 요약 출력
print(model_GT.summary())

                            OLS Regression Results                            
Dep. Variable:                  lb08m   R-squared:                       0.408
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     115.9
Date:                Fri, 06 Oct 2023   Prob (F-statistic):               0.00
Time:                        09:45:29   Log-Likelihood:            -1.2860e+05
No. Observations:               11824   AIC:                         2.573e+05
Df Residuals:                   11753   BIC:                         2.579e+05
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   9.563e+04    287.568    332.535      0.0

# Tabnet

In [126]:
def TabNet_modeling(X, y):
    # Road model
    reg = TabNetRegressor(optimizer_fn=torch.optim.Adam,
                          optimizer_params=dict(lr=1e-2),
                          scheduler_params={"step_size":50,"gamma":0.9},
                          scheduler_fn=torch.optim.lr_scheduler.StepLR,
                          mask_type='sparsemax', verbose=0, seed=2023)  # "sparsemax", entmax
    
    # Split data 
    X_train = X[X['date'] < 2306].drop('date', axis=1)
    X_test = X[X['date'] == 2306].drop('date', axis=1)
    y_train = y[X['date'] < 2306]
    y_test = y[X['date'] == 2306]

    # Further split training data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

    # Tunning
    reg.fit(X_train=X_train.values, y_train=y_train.values,
            eval_set=[(X_train.values, y_train.values), (X_val.values, y_val.values)],
            eval_name=['train', 'valid'],
            eval_metric=['rmse'],
            max_epochs=1000 , patience=300,
            batch_size=1024, virtual_batch_size=128,
            num_workers=0,
            drop_last=False)
    
    # Predict
    preds = reg.predict(X_test.values)
    test_mse = mean_squared_error(y_pred=preds, y_true=y_test)   
    
    # Selection
    fimportance = pd.DataFrame({'feature': X_train.columns, 'importance':reg.feature_importances_})
    unuse = fimportance.query('importance==0').feature.values.tolist()
    if len(unuse) != 0:
        X = X.drop(unuse, axis=1)
        return X, reg.best_cost, test_mse, reg
    else:
        return None, 'Done!', 'Done!', None


In [127]:
# X는 독립변수, y는 종속변수입니다.
X = df.drop([i for i in df.columns if 'lb08m' in i], axis=1)
y = df[[i for i in df.columns if 'lb08m' in i]]

In [ ]:
first_selected_feature, val_rmse, test_rmse, first_model = TabNet_modeling(X, y)
print(f"BEST VALID SCORE FOR DATA : {val_rmse}")
print(f"FINAL TEST SCORE FOR DATA : {test_rmse}")

In [129]:
def plot_feature_importances(fimportance):
    plt.figure(figsize=(10, 6))
    sns.barplot(x='importance', y='feature', data=fimportance.sort_values(by='importance', ascending=False))
    plt.title('Feature Importances')
    plt.show()

In [ ]:
# 피처 중요도 확인 (만약 모델이 학습이 완료되었다면)
if reg is not None:
    fimportance = pd.DataFrame({'feature': first_selected_feature.columns, 'importance': first_model.feature_importances_})
    print(fimportance.sort_values(by='importance', ascending=False))  # 피처 중요도 출력
    
    # 피처 중요도 시각화
    plot_feature_importances(fimportance)